In [1]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, KFold
from sklearn.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import wandb
import itertools
from random import shuffle

In [2]:
df = pd.read_csv('ratings.csv')

display(df.head())

print(df.dtypes)

,user_id,parent_asin,rating
0,AGKHLEW2SOWHNMFQIJGBECAF7INQ,B00YQ6X8EO,5.0
1,AGKHLEW2SOWHNMFQIJGBECAF7INQ,B081TJ8YS3,4.0
2,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,B097R46CSY,5.0
3,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,B09JS339BZ,1.0
4,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,B08BZ63GMJ,5.0


user_id         object
parent_asin     object
rating         float64
dtype: object


In [ ]:
project="cse158-project-rating"
entity="jackjack170827-uc-san-diego"

train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

reader = Reader(rating_scale=(1, 5))
train_val_set = Dataset.load_from_df(train_val_df[['user_id', 'parent_asin', 'rating']], reader)

test_set = list(test_df.itertuples(index=False, name=None))

param_grid = {
    "n_factors": [50, 100, 200],
    "lr_all": [0.001, 0.005, 0.01],
    "reg_all": [0.02, 0.05, 0.1],
    "n_epochs": [20, 50, 100, 200]
}

param_combinations = list(itertools.product(
    param_grid["n_factors"], param_grid["lr_all"],
    param_grid["reg_all"], param_grid["n_epochs"]
))

kf = KFold(n_splits=5)
best_params = None
best_val_rmse = float("inf")

for n_factors, lr_all, reg_all, n_epochs in param_combinations:
    config = {
        "n_factors": n_factors,
        "lr_all": lr_all,
        "reg_all": reg_all,
        "n_epochs": n_epochs
    }

    run = wandb.init(
        project=project,
        entity=entity,
        name=f"SVD_f{n_factors}_lr{lr_all}_reg{reg_all}_ep{n_epochs}",
        config=config
    )

    svd = SVD(**config)

    results = cross_validate(svd, train_val_set, measures=["RMSE", "MAE"], cv=kf)

    for fold_idx in range(kf.n_splits):
        wandb.log({
            "fold_val_MSE": results["test_rmse"][fold_idx] ** 2,
            "fold_val_RMSE": results["test_rmse"][fold_idx],
            "fold_val_MAE": results["test_mae"][fold_idx]
        })

    avg_val_mse = np.mean([rmse ** 2 for rmse in results["test_rmse"]])
    avg_val_rmse = np.mean(results["test_rmse"])
    avg_val_mae = np.mean(results["test_mae"])

    wandb.log({
        "avg_val_MSE": avg_val_mse,
        "avg_val_RMSE": avg_val_rmse,
        "avg_val_MAE": avg_val_mae
    })

    if avg_val_rmse < best_val_rmse:
        best_val_rmse = avg_val_rmse
        best_params = config

    run.finish()

print("Best Hyperparameters:", best_params)
    
svd = SVD(**best_params)
svd.fit(train_val_set.build_full_trainset())

predictions = svd.test(test_set)

test_mse = np.mean([(pred.est - pred.r_ui) ** 2 for pred in predictions])
test_rmse = np.sqrt(test_mse)
test_mae = np.mean([abs(pred.est - pred.r_ui) for pred in predictions])

test_run = wandb.init(
    project=project,
    entity=entity,
    name="Test_Set_Evaluation",
    config=best_params
)

wandb.log({
    "test_MSE": test_mse,
    "test_RMSE": test_rmse,
    "test_MAE": test_mae
})

test_run.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jackjack170827 (jackjack170827-uc-san-diego). Use `wandb login --relogin` to force relogin


avg_val_MAE,▁
avg_val_MSE,▁
avg_val_RMSE,▁
fold_val_MAE,▆▁▆▂█
fold_val_MSE,▇▁▇▄█
fold_val_RMSE,▇▁▇▄█
avg_val_MAE,1.19461
avg_val_MSE,2.07876
avg_val_RMSE,1.44179
fold_val_MAE,1.19686
fold_val_MSE,2.08649


avg_val_MAE,▁
avg_val_MSE,▁
avg_val_RMSE,▁
fold_val_MAE,▃█▆▁▅
fold_val_MSE,▄█▆▁▅
fold_val_RMSE,▄█▆▁▅
avg_val_MAE,1.17199
avg_val_MSE,2.02875
avg_val_RMSE,1.42434
fold_val_MAE,1.17223
fold_val_MSE,2.03022


avg_val_MAE,▁
avg_val_MSE,▁
avg_val_RMSE,▁
fold_val_MAE,▆▅▁▃█
fold_val_MSE,▆▆▁▃█
fold_val_RMSE,▆▆▁▃█
avg_val_MAE,1.1553
avg_val_MSE,1.99842
avg_val_RMSE,1.41365
fold_val_MAE,1.15727
fold_val_MSE,2.00679


avg_val_MAE,▁
avg_val_MSE,▁
avg_val_RMSE,▁
fold_val_MAE,▃▄▁█▅
fold_val_MSE,▂▂▁█▄
fold_val_RMSE,▂▂▁█▄
avg_val_MAE,1.14102
avg_val_MSE,1.98101
avg_val_RMSE,1.40748
fold_val_MAE,1.14156
fold_val_MSE,1.98245


...

avg_val_MAE,▁
avg_val_MSE,▁
avg_val_RMSE,▁
fold_val_MAE,▇█▁▅▂
fold_val_MSE,██▁▄▁
fold_val_RMSE,██▁▄▁
avg_val_MAE,1.14947
avg_val_MSE,1.9807
avg_val_RMSE,1.40737
fold_val_MAE,1.14808
fold_val_MSE,1.97264


avg_val_MAE,▁
avg_val_MSE,▁
avg_val_RMSE,▁
fold_val_MAE,█▅▁▄▆
fold_val_MSE,█▃▁▇▅
fold_val_RMSE,█▃▁▇▅
avg_val_MAE,1.15082
avg_val_MSE,1.98435
avg_val_RMSE,1.40867
fold_val_MAE,1.15121
fold_val_MSE,1.98422


avg_val_MAE,▁
avg_val_MSE,▁
avg_val_RMSE,▁
fold_val_MAE,▂▁▄▄█
fold_val_MSE,▂▁▂▇█
fold_val_RMSE,▂▁▂▇█
avg_val_MAE,1.15359
avg_val_MSE,1.9928
avg_val_RMSE,1.41166
fold_val_MAE,1.1566
fold_val_MSE,1.99908


Best Hyperparameters: {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.05, 'n_epochs': 50}


test_MAE,▁
test_MSE,▁
test_RMSE,▁
test_MAE,1.12818
test_MSE,1.95282
test_RMSE,1.39744
